# Score compression of resacled data (subsamples) with susequent neural density estimation

In [2]:
import numpy as np
import sys
# load your pydelfi version here (not necessary to include path if you have pip installed)
sys.path.append("/home/nessa/Documents/Projects/pydelfi/") 
import ndes.ndes as ndes
import delfi.delfi as delfi
import compression.score.score as score
import distributions.priors as priors
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)
%matplotlib inline

### load the data

In [54]:
# load data (note that order of params is not the same as data_sim)
data_scaled        = np.load('./data/data_scaled.npy')
data_scaled_full   = np.load('./data/data_scaled.full.npy')
data_scaled_cosmos = np.load('./data/params_conc.npy')
data_scaled_cosmos_full = np.load('./data/params_conc.full.npy')

covariance = np.load('./data/covariance.npy')

In [45]:
data_scaled.shape

(10100, 50)

In [34]:
# fiducial parameters (for compression)
index = 51
index_0 = 53
theta_fiducial = data_scaled_cosmos_full[index]
print(theta_fiducial)


[0.1 0.3 2.1]


In [35]:
theta_cov = data_scaled_cosmos_full[index_0]

### Covariance Matrix

In [36]:
# compute covariance from covariance sims
Cov_Inv = np.linalg.inv(covariance)

### Gaussian process interpolation of (precompressed) data

In [37]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C


kernel = C(1.0, (1e-4, 1e4)) * RBF(1, (1e-4, 1e4))
#Instanciate a Gaussian Process model
gp     = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10)


#  fit the Gaussian process model on the mean of the (precompressed) data for each comsologyy


# fit 
gp.fit(data_scaled_cosmos_full,data_scaled_full)

#use the GP prediction to build the model
def fitGP(theta):
    pred, stdev = gp.predict(np.array(theta).reshape(1,3),return_std=True)
    return pred.T

# compute derivative of mean at fiducial model with finite differencing

h      = 0.01

theta1 = np.asarray([theta_fiducial[0]*(1+h), theta_fiducial[1],theta_fiducial[2]])
theta1_= np.asarray([theta_fiducial[0]*(1-h), theta_fiducial[1],theta_fiducial[2]])

theta2 = np.asarray([theta_fiducial[0], theta_fiducial[1]*(1+h),theta_fiducial[2]])
theta2_= np.asarray([theta_fiducial[0], theta_fiducial[1]*(1-h),theta_fiducial[2]])

theta3 = np.asarray([theta_fiducial[0], theta_fiducial[1],theta_fiducial[2]*(1+h)])
theta3_= np.asarray([theta_fiducial[0], theta_fiducial[1],theta_fiducial[2]*(1-h)])

dmudt1 = (fitGP(theta1)-fitGP(theta1_))/(theta1-theta1_)[0]
dmudt2 = (fitGP(theta2)-fitGP(theta2_))/(theta2-theta2_)[1]
dmudt3 = (fitGP(theta3)-fitGP(theta3_))/(theta3-theta3_)[2]

# derivative
dmudt = np.hstack((dmudt1,dmudt2,dmudt3)).T

/home/nessa/miniconda2/envs/py3torch/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:480: ConvergenceWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-4.47331807e+06, -3.50756860e+00]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 70, 'nit': 25, 'warnflag': 2}
  ConvergenceWarning)


In [38]:
# set up scrore compression
mu             = fitGP(theta_fiducial)[:,0]
Cinv           = Cov_Inv


Compressor     = score.Gaussian(len(mu), theta_fiducial, mu = mu, Cinv = Cinv, dmudt = dmudt)
Compressor.compute_fisher()
Finv           = Compressor.Finv

def compressor(d, compressor_args):
    return Compressor.scoreMLE(d)
compressor_args=None

/home/nessa/miniconda2/envs/py3torch/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:357: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "


In [39]:
print(dmudt)

[[-2.72682977e+02 -4.71195347e+02 -6.42995065e+02 -9.62919499e+02
  -1.41649948e+03 -1.18779533e+03 -1.00374627e+03 -1.48568129e+03
  -1.37401463e+03 -1.26440719e+03 -6.98956597e+01 -4.93843221e+01
   1.17071695e+02  1.51716015e+03  1.04991881e+03  4.68832532e+03
   4.20764209e+03  4.96445396e+03  4.17861292e+03  4.16340762e+03
   5.50731383e+03  4.62079307e+03  5.06053439e+03  3.59664099e+03
   3.05866137e+03  2.77886747e+03  3.39388673e+03  8.28776873e+02
   2.88783664e+03  8.83308137e+02  1.67745933e+02  1.03581910e+03
  -3.15588527e+02 -1.57928029e+03 -1.67802068e+02 -1.30339485e+03
  -1.89796127e+03 -1.28076694e+03 -2.01318653e+03 -5.00280055e+02
  -9.45159323e+02 -1.67793670e+03 -1.08845833e+03 -7.81504647e+02
  -1.40226600e+03 -9.82106612e+02 -8.57388467e+02 -7.66056099e+02
  -6.20697591e+02 -1.30142668e+03]
 [ 6.88851085e+03  9.40667988e+03  9.56561959e+03  1.20485990e+04
   1.53294712e+04  1.58265048e+04  1.55156401e+04  1.53533591e+04
   1.01188757e+04  2.78593311e+03 -3.0146

In [76]:
# compress all the (precompressed data) with compressor
compressed_train = np.zeros((data_scaled.shape[0],3))
compressed_train = np.reshape(compressed_train,newshape=(101,100,3))
data_scaled_c    = np.reshape(data_scaled_cosmos,newshape=(101,100,3))
data_scaled_= np.reshape(data_scaled,newshape=(101,100,50))
# loop over cosmologies
for ii in range(compressed_train.shape[0]):
    for jj in range(compressed_train.shape[1]):
        compressed_train[ii][jj] = compressor(data_scaled_[ii][jj],None)
print(compressed_train.shape)

(101, 100, 3)


In [81]:
print(index)
compressed_train1=compressed_train[0:index,:,:]
print(compressed_train1.shape)
compressed_train2=compressed_train[index+1:,:,:]
compressed_data  =compressed_train[index,:,:]
compressed_train_= np.concatenate((compressed_train1,compressed_train2))
compressed_train_.shape, compressed_data.shape

data_scaled_c1=data_scaled_c[0:index,:,:]
print(compressed_train1.shape)
data_scaled_c2=data_scaled_c[index+1:,:,:]
data_scaled_c_data  =data_scaled_c[index,:,:]
data_scaled_c_= np.concatenate((data_scaled_c1,data_scaled_c2))
data_scaled_c_.shape, data_scaled_c_data.shape
#print(data_scaled_c_data)

51
(51, 100, 3)
(51, 100, 3)
[[0.1 0.3 2.1]
 [0.1 0.3 2.1]
 [0.1 0.3 2.1]
 [0.1 0.3 2.1]
 [0.1 0.3 2.1]
 [0.1 0.3 2.1]
 [0.1 0.3 2.1]
 [0.1 0.3 2.1]
 [0.1 0.3 2.1]
 [0.1 0.3 2.1]
 [0.1 0.3 2.1]
 [0.1 0.3 2.1]
 [0.1 0.3 2.1]
 [0.1 0.3 2.1]
 [0.1 0.3 2.1]
 [0.1 0.3 2.1]
 [0.1 0.3 2.1]
 [0.1 0.3 2.1]
 [0.1 0.3 2.1]
 [0.1 0.3 2.1]
 [0.1 0.3 2.1]
 [0.1 0.3 2.1]
 [0.1 0.3 2.1]
 [0.1 0.3 2.1]
 [0.1 0.3 2.1]
 [0.1 0.3 2.1]
 [0.1 0.3 2.1]
 [0.1 0.3 2.1]
 [0.1 0.3 2.1]
 [0.1 0.3 2.1]
 [0.1 0.3 2.1]
 [0.1 0.3 2.1]
 [0.1 0.3 2.1]
 [0.1 0.3 2.1]
 [0.1 0.3 2.1]
 [0.1 0.3 2.1]
 [0.1 0.3 2.1]
 [0.1 0.3 2.1]
 [0.1 0.3 2.1]
 [0.1 0.3 2.1]
 [0.1 0.3 2.1]
 [0.1 0.3 2.1]
 [0.1 0.3 2.1]
 [0.1 0.3 2.1]
 [0.1 0.3 2.1]
 [0.1 0.3 2.1]
 [0.1 0.3 2.1]
 [0.1 0.3 2.1]
 [0.1 0.3 2.1]
 [0.1 0.3 2.1]
 [0.1 0.3 2.1]
 [0.1 0.3 2.1]
 [0.1 0.3 2.1]
 [0.1 0.3 2.1]
 [0.1 0.3 2.1]
 [0.1 0.3 2.1]
 [0.1 0.3 2.1]
 [0.1 0.3 2.1]
 [0.1 0.3 2.1]
 [0.1 0.3 2.1]
 [0.1 0.3 2.1]
 [0.1 0.3 2.1]
 [0.1 0.3 2.1]
 [0.1 0.3 2.1]
 [0.1 0.3 2

In [82]:
compressed_train_=np.reshape(compressed_train_,newshape=(-1,3))
data_scaled_c_=np.reshape(data_scaled_c_,newshape=(-1,3))

### NDE estimation

In [83]:
# set up priors
lower = np.array([np.min(data_scaled_cosmos[:,0]),np.min(data_scaled_cosmos[:,1]),np.min(data_scaled_cosmos[:,2])])
upper = np.array([np.max(data_scaled_cosmos[:,0]),np.max(data_scaled_cosmos[:,1]),np.max(data_scaled_cosmos[:,2])])
print(upper, lower)
prior = priors.Uniform(lower, upper)

[0.62036 0.4159  2.9114 ] [0.     0.1841 1.2886]


In [84]:
# NDEs you wanna train
NDEs = [ndes.ConditionalMaskedAutoregressiveFlow(n_parameters=3, n_data=3, n_hiddens=[50,50], n_mades=5, act_fun=tf.tanh, index=5)]

In [85]:
DelfiEnsemble = delfi.Delfi(compressed_data[0], prior, NDEs, 
                            Finv = Finv, 
                            theta_fiducial = theta_fiducial, 
                            param_limits = [lower, upper],
                            param_names = ['M_\nu', '\Omega_m', 'A_s'], 
                            results_dir = "./",
                            input_normalization="fisher")

In [86]:
DelfiEnsemble.load_simulations(compressed_train_,data_scaled_c_)

In [ ]:
DelfiEnsemble.fisher_pretraining()

In [ ]:
DelfiEnsemble.train_ndes()

In [ ]:
posterior_samples = DelfiEnsemble.emcee_sample()

In [ ]:
DelfiEnsemble.triangle_plot(samples=[posterior_samples])